# AWS IoT DR - Register PCA with IoT Core

Register the private CA in the primary and secondary region with AWS IoT Core.

## Libraries

In [ ]:
from OpenSSL import crypto, SSL
from os.path import exists, join
from os import makedirs
from shutil import copy
from time import time, gmtime, localtime, strftime
import boto3
import json
import time

## Shared variables

Import shared variables into this notebook.

In [ ]:
%store -r config
print("config: {}".format(json.dumps(config, indent=4, default=str)))

## Some handy functions

Generate a key and create a certificate signing request (CSR).

In [ ]:
def create_csr(pkey, subject, digest="sha256"):
    print("subject: {}".format(subject))
    req = crypto.X509Req()
    subj = req.get_subject()
    
    for i in ['C', 'ST', 'L', 'O', 'OU', 'CN']:
        if i in subject:
            setattr(subj, i, subject[i])

    req.set_pubkey(pkey)
    req.sign(pkey, digest)
    return req


def create_priv_key_and_csr(cert_dir, csr_file, key_file, subject):
    if not exists(cert_dir):
        print("creating directory: {}".format(cert_dir))
        makedirs(cert_dir)
        
    priv_key = crypto.PKey()
    priv_key.generate_key(crypto.TYPE_RSA, 2048)
    #print(crypto.dump_privatekey(crypto.FILETYPE_PEM, priv_key).decode('utf-8'))

    key_file = join(cert_dir, key_file)
    f = open(key_file,"w")
    f.write(crypto.dump_privatekey(crypto.FILETYPE_PEM, priv_key).decode('utf-8'))
    f.close()
    
    csr = create_csr(priv_key, subject)

    csr_file = join(cert_dir, csr_file)
    f= open(csr_file,"w")
    f.write(crypto.dump_certificate_request(crypto.FILETYPE_PEM, csr).decode('utf-8'))
    f.close()
    
    return crypto.dump_certificate_request(crypto.FILETYPE_PEM, csr)

## Boto3 client
Create a boto3 client for the acm-pca service endpoint.

In [ ]:
c_acm_pca = boto3.client('acm-pca', region_name = config['aws_region_pca'])

## PCA certificate and ARN
Get the root certificate and the ARN from your private CA. They are required to register your private CA with AWS IoT Core.

In [ ]:
response = c_acm_pca.list_certificate_authorities(MaxResults=50)

for ca in response['CertificateAuthorities']:
    #print(ca['CertificateAuthorityConfiguration']['Subject']['CommonName'])
    if ca['CertificateAuthorityConfiguration']['Subject']['CommonName'] == config['Sub_CN']:
        pca_arn = ca['Arn']
        break

print("pca_arn: {}\n".format(pca_arn))

response = c_acm_pca.get_certificate_authority_certificate(
    CertificateAuthorityArn = pca_arn
)
print("response: {}\n".format(json.dumps(response, indent=4, default=str)))
pca_certificate = response['Certificate']
print("pca_certificate:\n{}".format(pca_certificate))

## Register private CA
To register the private CA with AWS IoT Core you need to get a registration code. Then you create a certificate with the common name (CN) set to the registration code. This certificate will be used for the CA registration process.

The private CA will be registered with AWS IoT Core in the primary and secondary region.

In [ ]:
for aws_region in [config['aws_region_primary'], config['aws_region_secondary']]: 
    print("AWS REGION: {}".format(aws_region))
    c_iot = boto3.client('iot', region_name = aws_region)
    time.sleep(2)

    response = c_iot.get_registration_code()

    print("response: {}\n".format(json.dumps(response, indent=4, default=str)))
    registration_code = response['registrationCode']
    print("registration_code: {}\n".format(registration_code))

    verification_csr = create_priv_key_and_csr(config['PCA_directory'], 
                                               'registration_csr_{}.pem'.format(aws_region), 
                                               'registration_key_{}.pem'.format(aws_region), 
                                               {"CN": registration_code})
    print("verification_csr:\n{}\n".format(verification_csr))

    idempotency_token = 'registration_cert'
    response = c_acm_pca.issue_certificate(
        CertificateAuthorityArn = pca_arn,
        Csr = verification_csr,
        SigningAlgorithm = 'SHA256WITHRSA',
        Validity= {
            'Value': 365,
            'Type': 'DAYS'
        },
        IdempotencyToken = idempotency_token
    )

    print("response: {}\n".format(json.dumps(response, indent=4, default=str)))
    certificate_arn = response['CertificateArn']

    print("certificate_arn: {}\n".format(certificate_arn))

    waiter = c_acm_pca.get_waiter('certificate_issued')
    try:
        waiter.wait(
            CertificateAuthorityArn=pca_arn,
            CertificateArn=certificate_arn
        )
    except botocore.exceptions.WaiterError as e:
        raise Exception("waiter: {}".format(e))
        
    response = c_acm_pca.get_certificate(
        CertificateAuthorityArn = pca_arn,
        CertificateArn = certificate_arn
    )
    print("response: {}".format(response))
    registration_certificate = response['Certificate']

    print("pca_certificate:\n{}\n".format(pca_certificate))
    print("registration_certificate:\n{}\n".format(registration_certificate))
    
    file_registration_crt = join(config['PCA_directory'], 'registration_cert_{}.pem'.format(aws_region))
    f = open(file_registration_crt,"w")
    f.write(registration_certificate)
    f.close()

    response = c_iot.register_ca_certificate(
        caCertificate = pca_certificate,
        verificationCertificate = registration_certificate,
        setAsActive = True,
        allowAutoRegistration = True
    )

    print("response: {}\n".format(json.dumps(response, indent=4, default=str)))

    certificate_id = response['certificateId']
    print("certificate_id: {}\n".format(certificate_id))

    response = c_iot.describe_ca_certificate(
        certificateId = certificate_id
    )

    print("response: {}\n".format(json.dumps(response, indent=4, default=str)))